# Data Cleaning & Loading SQL

In [1]:
import pandas as pd
import numpy as np
import datetime
import csv

from DATA225utils import make_connection, dataframe_query

import warnings
warnings.filterwarnings("ignore")

In [2]:
# connecttomysql.ini configuration file connects the python to mysql 
# so that we can create the database through python as well.
conn = make_connection(config_file = 'configFiles/local_snps_db.ini')
cursor = conn.cursor()

In [3]:
games_details = pd.read_csv("data/games_details.csv", low_memory = False)

games = pd.read_csv("data/games.csv", low_memory = False)

players = pd.read_csv("data/players.csv", low_memory = False)

ranking = pd.read_csv("data/ranking.csv", low_memory = False)

teams_df = pd.read_csv("data/teams.csv", low_memory = False)

In [4]:
print(games_details.shape)
print(games.shape)
print(players.shape)
print(ranking.shape)
print(teams_df.shape)

(668628, 29)
(26651, 21)
(7228, 4)
(210342, 13)
(30, 14)


In [5]:
df_all = games_details.merge(players.drop_duplicates(), on=['PLAYER_ID'], 
                   how='left', indicator=True)
df_all[df_all['_merge'] == 'left_only']["PLAYER_ID"]

1          1631110
6          1630170
7          1630200
24         1631103
27         1629646
            ...   
1378970       2412
1378976       1711
1378977        762
1378984       1505
1378985        696
Name: PLAYER_ID, Length: 79093, dtype: int64

In [6]:
data = df_all[df_all['_merge'] == 'left_only']["PLAYER_ID"]
missingPlayers = pd.DataFrame(data = data)

In [7]:
missingPlayers.drop_duplicates(inplace = True)

In [8]:
from nba_api.stats.endpoints import CommonPlayerInfo

def getMissingPlayerInfo(player_id):
    return CommonPlayerInfo(player_id).get_data_frames()[0][['FIRST_NAME', \
                                                             'LAST_NAME', \
                                                             'TEAM_ID', \
                                                             'HEIGHT', \
                                                             'DRAFT_YEAR']]

In [9]:
missingPlayers.isna().sum()

PLAYER_ID    0
dtype: int64

In [10]:
import time

for i, player in missingPlayers.iterrows():
    try:
        playerInfo = getMissingPlayerInfo(int(float(player.PLAYER_ID)))        
        missingPlayers.loc[i, "FIRST_NAME"] = playerInfo["FIRST_NAME"][0]
        missingPlayers.loc[i, "LAST_NAME"] = playerInfo["LAST_NAME"][0]
        missingPlayers.loc[i, "TEAM_ID"] = playerInfo["TEAM_ID"][0]
        time.sleep(0.5)
    except:
        continue
    

In [11]:
missingPlayers.loc[:, "FULL_NAME"] = missingPlayers.FIRST_NAME+" "+missingPlayers.LAST_NAME

In [12]:
data = {"PLAYER_NAME": list(missingPlayers.FULL_NAME), \
       "TEAM_ID": list(missingPlayers.TEAM_ID), \
       "PLAYER_ID": list(missingPlayers.PLAYER_ID), \
       "SEASON": '2014'}

In [13]:
missingPlayers_new = pd.DataFrame(data)

In [14]:
missingPlayers_new.head()

PLAYER_NAME       TEAM_ID  PLAYER_ID SEASON
0   Jeremy Sochan  1.610613e+09    1631110   2014
1   Devin Vassell  1.610613e+09    1630170   2014
2       Tre Jones  1.610613e+09    1630200   2014
3  Malaki Branham  1.610613e+09    1631103   2014
4  Charles Bassey  1.610613e+09    1629646   2014

In [15]:
fullPlayers = pd.concat([players, missingPlayers_new])

In [16]:
games_details = games_details.drop_duplicates(subset = ["GAME_ID", "TEAM_ID", "PLAYER_ID"])
games = games.drop_duplicates(subset = "GAME_ID")
players = players.drop_duplicates(subset = ["PLAYER_ID", "TEAM_ID"])
ranking = ranking.drop_duplicates(subset = ["TEAM_ID", "STANDINGSDATE"])
teams_df = teams_df.drop_duplicates(subset = "TEAM_ID")

In [17]:
print(games_details.shape)
print(games.shape)
print(players.shape)
print(ranking.shape)
print(teams_df.shape)

(668339, 29)
(26622, 21)
(4281, 4)
(210313, 13)
(30, 14)


In [18]:
games_details.isna().sum()/games_details.shape[0]*100

GAME_ID               0.000000
TEAM_ID               0.000000
TEAM_ABBREVIATION     0.000000
TEAM_CITY             0.000000
PLAYER_ID             0.000000
PLAYER_NAME           0.000000
NICKNAME             92.064357
START_POSITION       61.749352
COMMENT              83.590962
MIN                  16.409188
FGM                  16.409188
FGA                  16.409188
FG_PCT               16.409188
FG3M                 16.409188
FG3A                 16.409188
FG3_PCT              16.409188
FTM                  16.409188
FTA                  16.409188
FT_PCT               16.409188
OREB                 16.409188
DREB                 16.409188
REB                  16.409188
AST                  16.409188
STL                  16.409188
BLK                  16.409188
TO                   16.409188
PF                   16.409188
PTS                  16.409188
PLUS_MINUS           19.949457
dtype: float64

### The missing stats means that the players simply didn't play in the match. So we will drop Nickname, start position an comment which has a lot of missing values and no importance in the analysis

In [19]:
games_details.drop(["NICKNAME", "START_POSITION", "COMMENT", "PLUS_MINUS"], axis = 1, inplace = True)

In [20]:
games.isna().sum()/games.shape[0]

GAME_DATE_EST       0.000000
GAME_ID             0.000000
GAME_STATUS_TEXT    0.000000
HOME_TEAM_ID        0.000000
VISITOR_TEAM_ID     0.000000
SEASON              0.000000
TEAM_ID_home        0.000000
PTS_home            0.003719
FG_PCT_home         0.003719
FT_PCT_home         0.003719
FG3_PCT_home        0.003719
AST_home            0.003719
REB_home            0.003719
TEAM_ID_away        0.000000
PTS_away            0.003719
FG_PCT_away         0.003719
FT_PCT_away         0.003719
FG3_PCT_away        0.003719
AST_away            0.003719
REB_away            0.003719
HOME_TEAM_WINS      0.000000
dtype: float64

In [21]:
games[games.PTS_away.isna()].GAME_DATE_EST.value_counts()

2003-10-14    10
2003-10-24     9
2003-10-11     9
2003-10-17     9
2003-10-22     8
2003-10-18     7
2003-10-16     6
2003-10-23     6
2003-10-21     5
2003-10-19     5
2003-10-10     5
2003-10-09     4
2003-10-15     4
2003-10-20     4
2003-10-12     3
2003-10-13     2
2003-10-08     2
2003-10-07     1
Name: GAME_DATE_EST, dtype: int64

### There is no data even on the official NBA site for these datapoints, as they are all concentrated in 2003 we will just drop 2003 data and start from 2004.

In [22]:
games.isna().sum()

GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64

In [23]:
games = games[games.GAME_DATE_EST>'2004-01-01']

In [24]:
players.isna().sum()/players.shape[0]*100

PLAYER_NAME    0.0
TEAM_ID        0.0
PLAYER_ID      0.0
SEASON         0.0
dtype: float64

### Doing the same for the ranking table as they are the only two tables with date 

In [25]:
ranking = ranking[ranking.STANDINGSDATE>'2004-01-01']

In [26]:
ranking.isna().sum()/ranking.shape[0]*100

TEAM_ID           0.000000
LEAGUE_ID         0.000000
SEASON_ID         0.000000
STANDINGSDATE     0.000000
CONFERENCE        0.000000
TEAM              0.000000
G                 0.000000
W                 0.000000
L                 0.000000
W_PCT             0.000000
HOME_RECORD       0.000000
ROAD_RECORD       0.000000
RETURNTOPLAY     98.078183
dtype: float64

In [27]:
ranking.drop("RETURNTOPLAY", axis = 1, inplace = True)

In [28]:
teams_df.isna().sum()/teams_df.shape[0]*100

LEAGUE_ID              0.000000
TEAM_ID                0.000000
MIN_YEAR               0.000000
MAX_YEAR               0.000000
ABBREVIATION           0.000000
NICKNAME               0.000000
YEARFOUNDED            0.000000
CITY                   0.000000
ARENA                  0.000000
ARENACAPACITY         13.333333
OWNER                  0.000000
GENERALMANAGER         0.000000
HEADCOACH              0.000000
DLEAGUEAFFILIATION     0.000000
dtype: float64

Let ArenaCapacity be null

## Loading Players

In [29]:
players.head()

PLAYER_NAME     TEAM_ID  PLAYER_ID  SEASON
0     Royce O'Neale  1610612762    1626220    2019
1  Bojan Bogdanovic  1610612762     202711    2019
2       Rudy Gobert  1610612762     203497    2019
3  Donovan Mitchell  1610612762    1628378    2019
4       Mike Conley  1610612762     201144    2019

### Adding more features for each player like height, weight, birthdate and draftyear.

In [30]:
from nba_api.stats.endpoints import commonplayerinfo

def getPlayerInfo(player_id):
    return commonplayerinfo.CommonPlayerInfo(player_id).get_data_frames()[0][['COUNTRY', 'HEIGHT',
       'WEIGHT', 'DRAFT_YEAR']]

In [31]:
player_wo_duplicates = fullPlayers.drop_duplicates(subset = "PLAYER_ID")

In [32]:
player_wo_duplicates.loc[:, 'COUNTRY'] = np.nan
player_wo_duplicates.loc[:, 'HEIGHT'] = np.nan
player_wo_duplicates.loc[:, 'WEIGHT'] = np.nan
player_wo_duplicates.loc[:, 'DRAFT_YEAR'] = np.nan

In [33]:
import time
for i in player_wo_duplicates.PLAYER_ID:
    try:
        df = getPlayerInfo(i)
        player_wo_duplicates.loc[player_wo_duplicates.PLAYER_ID==i, ['COUNTRY']] = df.COUNTRY[0]
        player_wo_duplicates.loc[player_wo_duplicates.PLAYER_ID==i, ['HEIGHT']] = df.HEIGHT[0]
        player_wo_duplicates.loc[player_wo_duplicates.PLAYER_ID==i, ['WEIGHT']] = df.WEIGHT[0]
        player_wo_duplicates.loc[player_wo_duplicates.PLAYER_ID==i, ['DRAFT_YEAR']] = df.DRAFT_YEAR[0]
        time.sleep(0.5)
    except:
        continue

In [34]:
player_wo_duplicates.isna().sum()

PLAYER_NAME     64
TEAM_ID         64
PLAYER_ID        0
SEASON           0
COUNTRY        231
HEIGHT          87
WEIGHT          87
DRAFT_YEAR      87
dtype: int64

In [35]:
# new_players = pd.read_csv("players_new.csv", low_memory = False)

In [36]:
# for i, player in players.iterrows():
#     players.loc[i, "COUNTRY"] = list(new_players[new_players.PLAYER_ID==player.PLAYER_ID]["COUNTRY"])[0]
#     players.loc[i, "HEIGHT"] = list(new_players[new_players.PLAYER_ID==player.PLAYER_ID]["HEIGHT"])[0]
#     players.loc[i, "WEIGHT"] = list(new_players[new_players.PLAYER_ID==player.PLAYER_ID]["WEIGHT"])[0]
#     players.loc[i, "DRAFT_YEAR"] = list(new_players[new_players.PLAYER_ID==player.PLAYER_ID]["DRAFT_YEAR"])[0]

In [37]:
# players.to_csv("players_new.csv")

In [38]:
def convertHeight(heightInFoot):
    if len(str(heightInFoot).split("-")) == 2:
        return(float(heightInFoot.split("-")[0]+"."+player.HEIGHT.split("-")[1]))*30.48
    else:
        return(float(heightInFoot)*30.48)

In [39]:
for i, player in player_wo_duplicates.iterrows():
    if player.HEIGHT != '':
        if len(str(player.HEIGHT).split("-")) == 2:
            player_wo_duplicates.loc[i, "HEIGHT"] = convertHeight(player.HEIGHT)
        else:
            player_wo_duplicates.loc[i, "HEIGHT"] = convertHeight(player.HEIGHT)
    else:
        player_wo_duplicates.loc[i, "HEIGHT"] = np.nan

In [40]:
player_wo_duplicates.loc[:, "COUNTRY"] = player_wo_duplicates.COUNTRY.apply(lambda x: None if x == '' else x)
player_wo_duplicates.loc[:, "HEIGHT"] = player_wo_duplicates.HEIGHT.apply(lambda x: None if x == '' else x)
player_wo_duplicates.loc[:, "WEIGHT"] = player_wo_duplicates.WEIGHT.apply(lambda x: None if x == '' else x)
player_wo_duplicates.loc[:, "DRAFT_YEAR"] = player_wo_duplicates.DRAFT_YEAR.apply(lambda x: None if x == '' else x)

In [41]:
cursor.execute("""DROP TABLE IF EXISTS players""")

sql_create = ("""
                CREATE TABLE players
                (
                  player_id INT NOT NULL,
                  player_name VARCHAR(50),
                  country VARCHAR(255),
                  height INT,
                  weight INT,
                  draftyear VARCHAR(10),
                  PRIMARY KEY (player_id)
                )
              """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO players
            VALUES (%s, %s, %s, %s, %s, %s)
            """
        )

for index, row in player_wo_duplicates.iterrows():
    values = (row['PLAYER_ID'], row['PLAYER_NAME'], \
              row['COUNTRY'], row['HEIGHT'], row['WEIGHT'], \
              row['DRAFT_YEAR'])
    cursor.execute(sql_insert, values)
    
conn.commit()

## Loading Teams

In [42]:
teams_df.head()

LEAGUE_ID     TEAM_ID  MIN_YEAR  MAX_YEAR ABBREVIATION   NICKNAME  \
0          0  1610612737      1949      2019          ATL      Hawks   
1          0  1610612738      1946      2019          BOS    Celtics   
2          0  1610612740      2002      2019          NOP   Pelicans   
3          0  1610612741      1966      2019          CHI      Bulls   
4          0  1610612742      1980      2019          DAL  Mavericks   

   YEARFOUNDED         CITY                     ARENA  ARENACAPACITY  \
0         1949      Atlanta          State Farm Arena        18729.0   
1         1946       Boston                 TD Garden        18624.0   
2         2002  New Orleans      Smoothie King Center            NaN   
3         1966      Chicago             United Center        21711.0   
4         1980       Dallas  American Airlines Center        19200.0   

             OWNER  GENERALMANAGER      HEADCOACH DLEAGUEAFFILIATION  
0     Tony Ressler  Travis Schlenk   Lloyd Pierce      Erie Bayhawks  
1    Wyc Grousbeck     Danny Ainge   Brad Stevens    Maine Red Claws  
2       Tom Benson  Trajan Langdon   Alvin Gentry       No Affiliate  
3  Jerry Reinsdorf      Gar Forman     Jim Boylen   Windy City Bulls  
4       Mark Cuban   Donnie Nelson  Rick Carlisle      Texas Legends

### Changing New Orleans Pelicans conference to West (it changed from East to West in the 2004 season)

In [43]:
ranking.loc[ranking.TEAM_ID==1610612740,'CONFERENCE'] = "West"

In [44]:
teams_df = pd.merge(teams_df,ranking[["TEAM_ID", "CONFERENCE"]].drop_duplicates(), on = "TEAM_ID", how='left')

In [45]:
from nba_api.stats.static import teams

teams_df["STATE"] = teams_df.NICKNAME.apply(lambda x: teams.find_teams_by_nickname(x)[0]["state"])

In [46]:
teams_df.head()

LEAGUE_ID     TEAM_ID  MIN_YEAR  MAX_YEAR ABBREVIATION   NICKNAME  \
0          0  1610612737      1949      2019          ATL      Hawks   
1          0  1610612738      1946      2019          BOS    Celtics   
2          0  1610612740      2002      2019          NOP   Pelicans   
3          0  1610612741      1966      2019          CHI      Bulls   
4          0  1610612742      1980      2019          DAL  Mavericks   

   YEARFOUNDED         CITY                     ARENA  ARENACAPACITY  \
0         1949      Atlanta          State Farm Arena        18729.0   
1         1946       Boston                 TD Garden        18624.0   
2         2002  New Orleans      Smoothie King Center            NaN   
3         1966      Chicago             United Center        21711.0   
4         1980       Dallas  American Airlines Center        19200.0   

             OWNER  GENERALMANAGER      HEADCOACH DLEAGUEAFFILIATION  \
0     Tony Ressler  Travis Schlenk   Lloyd Pierce      Erie Bayhawks   
1    Wyc Grousbeck     Danny Ainge   Brad Stevens    Maine Red Claws   
2       Tom Benson  Trajan Langdon   Alvin Gentry       No Affiliate   
3  Jerry Reinsdorf      Gar Forman     Jim Boylen   Windy City Bulls   
4       Mark Cuban   Donnie Nelson  Rick Carlisle      Texas Legends   

  CONFERENCE          STATE  
0       East        Georgia  
1       East  Massachusetts  
2       West      Louisiana  
3       East       Illinois  
4       West          Texas

In [47]:
sql_create = ("""
                CREATE TABLE teams
                (
                  team_id INT NOT NULL,
                  abbrevation VARCHAR(3) NOT NULL,
                  name VARCHAR(50) NOT NULL,
                  year_founded INT NOT NULL,
                  city VARCHAR(50) NOT NULL,
                  arena VARCHAR(50) NOT NULL,
                  arena_capacity INT,
                  owner VARCHAR(50) NOT NULL,
                  general_manager VARCHAR(50) NOT NULL,
                  head_coach VARCHAR(50) NOT NULL,
                  d_league_affiliation VARCHAR(50) NOT NULL,
                  state VARCHAR(50) NOT NULL,
                  conference VARCHAR(4) NOT NULL,
                  PRIMARY KEY (team_id)
                )
              """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO teams
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        )

for index, row in teams_df.iterrows():
    values = (row['TEAM_ID'], row['ABBREVIATION'], row['NICKNAME'], \
              row['YEARFOUNDED'], row['CITY'], row['ARENA'], \
              row['ARENACAPACITY'], row['OWNER'], row['GENERALMANAGER'],
              row['HEADCOACH'], row['DLEAGUEAFFILIATION'], row['STATE'],
              row['CONFERENCE'])
    cursor.execute(sql_insert, values)
    
conn.commit()

## Loading Ranking

In [48]:
ranking.head()

TEAM_ID  LEAGUE_ID  SEASON_ID STANDINGSDATE CONFERENCE         TEAM   G  \
0  1610612743          0      22022    2022-12-22       West       Denver  30   
1  1610612763          0      22022    2022-12-22       West      Memphis  30   
2  1610612740          0      22022    2022-12-22       West  New Orleans  31   
3  1610612756          0      22022    2022-12-22       West      Phoenix  32   
4  1610612746          0      22022    2022-12-22       West  LA Clippers  33   

    W   L  W_PCT HOME_RECORD ROAD_RECORD  
0  19  11  0.633        10-3         9-8  
1  19  11  0.633        13-2         6-9  
2  19  12  0.613        13-4         6-8  
3  19  13  0.594        14-4         5-9  
4  19  14  0.576        11-7         8-7

In [49]:
ranking[["STARTDATE"]] = np.nan

In [50]:
for i in ranking.SEASON_ID.unique():
    ranking.loc[ranking.SEASON_ID==i, ['STARTDATE']] = min(ranking[ranking.SEASON_ID==i].STANDINGSDATE)

In [51]:
ranking.head()

TEAM_ID  LEAGUE_ID  SEASON_ID STANDINGSDATE CONFERENCE         TEAM   G  \
0  1610612743          0      22022    2022-12-22       West       Denver  30   
1  1610612763          0      22022    2022-12-22       West      Memphis  30   
2  1610612740          0      22022    2022-12-22       West  New Orleans  31   
3  1610612756          0      22022    2022-12-22       West      Phoenix  32   
4  1610612746          0      22022    2022-12-22       West  LA Clippers  33   

    W   L  W_PCT HOME_RECORD ROAD_RECORD   STARTDATE  
0  19  11  0.633        10-3         9-8  2022-10-18  
1  19  11  0.633        13-2         6-9  2022-10-18  
2  19  12  0.613        13-4         6-8  2022-10-18  
3  19  13  0.594        14-4         5-9  2022-10-18  
4  19  14  0.576        11-7         8-7  2022-10-18

In [52]:
cursor.execute("""DROP TABLE IF EXISTS ranking""")

sql_create = ( """
                CREATE TABLE ranking
                (
                  standings_date DATE NOT NULL,
                  GP INT NOT NULL,
                  GW INT NOT NULL,
                  GL INT NOT NULL,
                  home_record VARCHAR(5) NOT NULL,
                  away_record VARCHAR(5) NOT NULL,
                  conference VARCHAR(4) NOT NULL,
                  team_id INT NOT NULL,
                  PRIMARY KEY (standings_date, team_id),
                  FOREIGN KEY (team_id) REFERENCES teams(team_id)
                )
                """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO ranking
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
        )

for index, row in ranking.iterrows():
    values = (row['STANDINGSDATE'], row['G'], row['W'], \
              row['L'], row['HOME_RECORD'], row['ROAD_RECORD'], \
              row['CONFERENCE'], row['TEAM_ID'])
    cursor.execute(sql_insert, values)
    
conn.commit()

## Loading roster

In [53]:
players[players.PLAYER_NAME == "Kevin Durant"]

PLAYER_NAME     TEAM_ID  PLAYER_ID  SEASON
484   Kevin Durant  1610612751     201142    2019
652   Kevin Durant  1610612744     201142    2018
2799  Kevin Durant  1610612760     201142    2015

In [54]:
players

PLAYER_NAME     TEAM_ID  PLAYER_ID  SEASON
0        Royce O'Neale  1610612762    1626220    2019
1     Bojan Bogdanovic  1610612762     202711    2019
2          Rudy Gobert  1610612762     203497    2019
3     Donovan Mitchell  1610612762    1628378    2019
4          Mike Conley  1610612762     201144    2019
...                ...         ...        ...     ...
7223       Lanny Smith  1610612758     201831    2009
7224     Warren Carter  1610612752     201999    2009
7225      Bennet Davis  1610612751     201834    2009
7226    Brian Hamilton  1610612751     201646    2009
7227   Wesley Matthews  1610612762     500032    2009

[4281 rows x 4 columns]

In [55]:
fullPlayers = fullPlayers.drop_duplicates(subset = ["PLAYER_ID", "TEAM_ID"])

In [56]:
fullPlayers.dropna(inplace = True)

In [57]:
fullPlayers = fullPlayers[fullPlayers.TEAM_ID != 0]

In [58]:
cursor.execute("""DROP TABLE IF EXISTS roster""")

sql_create = ( """
                CREATE TABLE roster
                (
                  season INT NOT NULL,
                  team_id INT NOT NULL,
                  player_id INT NOT NULL,
                  PRIMARY KEY (team_id, player_id),
                  FOREIGN KEY (team_id) REFERENCES teams(team_id),
                  FOREIGN KEY (player_id) REFERENCES players(player_id),
                  UNIQUE (team_id, player_id)
                )
                """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO roster
            VALUES (%s, %s, %s)
            """
        )

for index, row in fullPlayers.iterrows():
    values = (row['SEASON'], row['TEAM_ID'], row['PLAYER_ID'])
    cursor.execute(sql_insert, values)
    
conn.commit()

## Loading Games

In [59]:
games.head()

GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0    2022-12-22  22200477            Final    1610612740       1610612759   
1    2022-12-22  22200478            Final    1610612762       1610612764   
2    2022-12-21  22200466            Final    1610612739       1610612749   
3    2022-12-21  22200467            Final    1610612755       1610612765   
4    2022-12-21  22200468            Final    1610612737       1610612741   

   SEASON  TEAM_ID_home  PTS_home  FG_PCT_home  FT_PCT_home  ...  AST_home  \
0    2022    1610612740     126.0        0.484        0.926  ...      25.0   
1    2022    1610612762     120.0        0.488        0.952  ...      16.0   
2    2022    1610612739     114.0        0.482        0.786  ...      22.0   
3    2022    1610612755     113.0        0.441        0.909  ...      27.0   
4    2022    1610612737     108.0        0.429        1.000  ...      22.0   

   REB_home  TEAM_ID_away  PTS_away  FG_PCT_away  FT_PCT_away  FG3_PCT_away  \
0      46.0    1610612759     117.0        0.478        0.815         0.321   
1      40.0    1610612764     112.0        0.561        0.765         0.333   
2      37.0    1610612749     106.0        0.470        0.682         0.433   
3      49.0    1610612765      93.0        0.392        0.735         0.261   
4      47.0    1610612741     110.0        0.500        0.773         0.292   

   AST_away  REB_away  HOME_TEAM_WINS  
0      23.0      44.0               1  
1      20.0      37.0               1  
2      20.0      46.0               1  
3      15.0      46.0               1  
4      20.0      47.0               0  

[5 rows x 21 columns]

In [60]:
cursor.execute("""DROP TABLE IF EXISTS games""")

sql_create = ( """
                CREATE TABLE games
                (
                  game_date DATE NOT NULL,
                  game_id INT NOT NULL,
                  status VARCHAR(5) NOT NULL,
                  season INT NOT NULL,
                  home_team_wins INT NOT NULL,
                  home_team_id INT NOT NULL,
                  visitor_team_id INT NOT NULL,
                  PRIMARY KEY (game_id),
                  FOREIGN KEY (home_team_id) REFERENCES teams(team_id),
                  FOREIGN KEY (visitor_team_id) REFERENCES teams(team_id)
                )
                """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO games
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
        )

for index, row in games.iterrows():
    values = (row['GAME_DATE_EST'], row['GAME_ID'], row['GAME_STATUS_TEXT'],\
              row['SEASON'], row['HOME_TEAM_WINS'],row['HOME_TEAM_ID'], \
              row['VISITOR_TEAM_ID'])
    cursor.execute(sql_insert, values)
    
conn.commit()

In [61]:
cursor.execute("""DROP TABLE IF EXISTS game_stats""")

sql_create = ( """
                CREATE TABLE game_stats
                (
                  date DATE NOT NULL,
                  game_id INT NOT NULL,
                  PTS INT NOT NULL,
                  FG_PCT FLOAT NOT NULL,
                  FT_PCT FLOAT NOT NULL,
                  FG3_PCT FLOAT NOT NULL,
                  AST INT NOT NULL,
                  REB INT NOT NULL,
                  team_id INT NOT NULL,
                  PRIMARY KEY (game_id, team_id),
                  FOREIGN KEY (team_id) REFERENCES teams(team_id)
                )
                """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO game_stats
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        )

for index, row in games.iterrows():
    values = (row['GAME_DATE_EST'], row['GAME_ID'], row['PTS_home'], \
              row['FG_PCT_home'], row['FT_PCT_home'], row['FG3_PCT_home'], \
              row['AST_home'], row['REB_home'], row['TEAM_ID_home'])
    cursor.execute(sql_insert, values)

for index, row in games.iterrows():
    values = (row['GAME_DATE_EST'], row['GAME_ID'], row['PTS_away'], \
              row['FG_PCT_away'], row['FT_PCT_away'], row['FG3_PCT_away'], \
              row['AST_away'], row['REB_away'], row['TEAM_ID_away'])
    cursor.execute(sql_insert, values)
    
conn.commit()

## Loading Games Details

In [62]:
games_details.head()

GAME_ID     TEAM_ID TEAM_ABBREVIATION    TEAM_CITY  PLAYER_ID  \
0  22200477  1610612759               SAS  San Antonio    1629641   
1  22200477  1610612759               SAS  San Antonio    1631110   
2  22200477  1610612759               SAS  San Antonio    1627751   
3  22200477  1610612759               SAS  San Antonio    1630170   
4  22200477  1610612759               SAS  San Antonio    1630200   

      PLAYER_NAME    MIN  FGM   FGA  FG_PCT  ...  FT_PCT  OREB  DREB  REB  \
0  Romeo Langford  18:06  1.0   1.0   1.000  ...     0.0   1.0   1.0  2.0   
1   Jeremy Sochan  31:01  7.0  14.0   0.500  ...     0.7   6.0   3.0  9.0   
2    Jakob Poeltl  21:42  6.0   9.0   0.667  ...     1.0   1.0   3.0  4.0   
3   Devin Vassell  30:20  4.0  13.0   0.308  ...     1.0   0.0   9.0  9.0   
4       Tre Jones  27:44  7.0  12.0   0.583  ...     1.0   0.0   2.0  2.0   

   AST  STL  BLK   TO   PF   PTS  
0  0.0  1.0  0.0  2.0  5.0   2.0  
1  6.0  1.0  0.0  2.0  1.0  23.0  
2  1.0  1.0  0.0  2.0  4.0  13.0  
3  5.0  3.0  0.0  2.0  1.0  10.0  
4  3.0  0.0  0.0  2.0  2.0  19.0  

[5 rows x 25 columns]

In [63]:
def convert_to_sec(x):
    if pd.isnull(x):
        return x
    else:
        if len(x.split(":")) > 1:
            return int(float(x.split(":")[0]))*60+int(float(x.split(":")[1]))
        else:
            return int(float(x))*60

In [64]:
games_details["seconds"] = games_details.MIN.apply(lambda x: convert_to_sec(x))

In [65]:
games_details.isna().sum()

GAME_ID                   0
TEAM_ID                   0
TEAM_ABBREVIATION         0
TEAM_CITY                 0
PLAYER_ID                 0
PLAYER_NAME               0
MIN                  109669
FGM                  109669
FGA                  109669
FG_PCT               109669
FG3M                 109669
FG3A                 109669
FG3_PCT              109669
FTM                  109669
FTA                  109669
FT_PCT               109669
OREB                 109669
DREB                 109669
REB                  109669
AST                  109669
STL                  109669
BLK                  109669
TO                   109669
PF                   109669
PTS                  109669
seconds              109669
dtype: int64

In [66]:
df_all = games_details.merge(games.drop_duplicates(), on=['GAME_ID'], 
                   how='left', indicator=True)
data = df_all[df_all['_merge'] == 'left_only']["GAME_ID"].unique()

In [67]:
new_games_details = games_details[~games_details['GAME_ID'].isin(data)]

In [68]:
new_games_details

GAME_ID     TEAM_ID TEAM_ABBREVIATION    TEAM_CITY  PLAYER_ID  \
0       22200477  1610612759               SAS  San Antonio    1629641   
1       22200477  1610612759               SAS  San Antonio    1631110   
2       22200477  1610612759               SAS  San Antonio    1627751   
3       22200477  1610612759               SAS  San Antonio    1630170   
4       22200477  1610612759               SAS  San Antonio    1630200   
...          ...         ...               ...          ...        ...   
668623  11200005  1610612743               DEN       Denver     202706   
668624  11200005  1610612743               DEN       Denver     202702   
668625  11200005  1610612743               DEN       Denver     201585   
668626  11200005  1610612743               DEN       Denver     202389   
668627  11200005  1610612743               DEN       Denver     201951   

            PLAYER_NAME    MIN  FGM   FGA  FG_PCT  ...  OREB  DREB  REB  AST  \
0        Romeo Langford  18:06  1.0   1.0   1.000  ...   1.0   1.0  2.0  0.0   
1         Jeremy Sochan  31:01  7.0  14.0   0.500  ...   6.0   3.0  9.0  6.0   
2          Jakob Poeltl  21:42  6.0   9.0   0.667  ...   1.0   3.0  4.0  1.0   
3         Devin Vassell  30:20  4.0  13.0   0.308  ...   0.0   9.0  9.0  5.0   
4             Tre Jones  27:44  7.0  12.0   0.583  ...   0.0   2.0  2.0  3.0   
...                 ...    ...  ...   ...     ...  ...   ...   ...  ...  ...   
668623  Jordan Hamilton     19  4.0   9.0   0.444  ...   0.0   2.0  2.0  0.0   
668624   Kenneth Faried     23  7.0  11.0   0.636  ...   1.0   0.0  1.0  1.0   
668625     Kosta Koufos     15  3.0   7.0   0.429  ...   3.0   5.0  8.0  0.0   
668626   Timofey Mozgov     19  1.0   1.0   1.000  ...   1.0   2.0  3.0  1.0   
668627        Ty Lawson     27  3.0   6.0   0.500  ...   0.0   2.0  2.0  6.0   

        STL  BLK   TO   PF   PTS  seconds  
0       1.0  0.0  2.0  5.0   2.0   1086.0  
1       1.0  0.0  2.0  1.0  23.0   1861.0  
2       1.0  0.0  2.0  4.0  13.0   1302.0  
3       3.0  0.0  2.0  1.0  10.0   1820.0  
4       0.0  0.0  2.0  2.0  19.0   1664.0  
...     ...  ...  ...  ...   ...      ...  
668623  2.0  0.0  1.0  3.0  17.0   1140.0  
668624  1.0  0.0  3.0  3.0  18.0   1380.0  
668625  1.0  0.0  0.0  3.0   6.0    900.0  
668626  0.0  0.0  4.0  2.0   2.0   1140.0  
668627  2.0  0.0  6.0  1.0   8.0   1620.0  

[657203 rows x 26 columns]

In [69]:
cursor.execute("""DROP TABLE IF EXISTS player_game_stats""")

sql_create = ( """
                CREATE TABLE player_game_stats
                (
                  team_id INT NOT NULL,
                  SECONDS INT,
                  FGM INT,
                  FGA INT,
                  FG3M INT,
                  FG3A INT,
                  FTM INT,
                  FTA INT,
                  OREB INT,
                  DREB INT,
                  AST INT,
                  STL INT,
                  BLK INT,
                  TurnOver INT,
                  PF INT,
                  PTS INT,
                  REB INT,
                  game_id INT NOT NULL,
                  player_id INT NOT NULL,
                  PRIMARY KEY (team_id, game_id, player_id),
                  FOREIGN KEY (game_id) REFERENCES games(game_id),
                  FOREIGN KEY (player_id) REFERENCES players(player_id)
                )
                """)

cursor.execute(sql_create);

sql_insert = (   """
            INSERT INTO player_game_stats
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        )

for index, row in new_games_details.iterrows():
    values = (row['TEAM_ID'], row['seconds'], row['FGM'], \
              row['FGA'], row['FG3M'], row['FG3A'], \
              row['FTM'], row['FTA'], row['OREB'], \
              row['DREB'], row['AST'], row['STL'], \
              row['BLK'], row['TO'], row['PF'], \
              row['PTS'], row['REB'], \
              row['GAME_ID'], row['PLAYER_ID'])
    cursor.execute(sql_insert, values)
    
conn.commit()

In [70]:
cursor.close()
conn.close()